In [126]:
from bs4 import BeautifulSoup
import polars as pl
from markdownify import markdownify as md
import stanza
import dspy
from dotenv import load_dotenv
import os
from spacy_conll import init_parser
import spacy
load_dotenv()

True

### Loading from Google Doc Table

Doc was manually exported as html and saved locally. Html was then parsed and both text version converted to a table

In [116]:
with open("data/doc.html") as f:
    content = f.read()
soup = BeautifulSoup(content)

In [117]:
table = soup.find_all("table")[0]
all_cells = table.find_all("td")

second_col = []
fourth_col = []

for row in table.find_all("tr"):
    cells = row.find_all("td")

    if len(cells) < 4:
        continue

    def proc(cell):
        ps = cell.find_all("p")
        lst = []
        for p in ps:
            if p.text == "":
                # replace empty paragraph with br tag
                lst.append("<br>")
            else:
                lst.append(p.decode_contents().strip())
        temp = " ".join(lst)

        temp = md(temp).replace("\n   \n", "")
        return temp.strip()

    second_col.append(proc(cells[1]))
    fourth_col.append(proc(cells[3]))

data = {"t1": second_col[1:], "t2": fourth_col[1:]}
df = pl.from_dict(data)
example = df[0]["t1"].item()
example

'Die Männerrunde am Brunnen   \n Herr Jakob steht am Brunnen und beobachtet, wie die andere Männer ihre kleine Schiffe zu Wasser lassen. Er geht und holt Materialien für sein Schiff. Er bastelt sein Schiff. Herr Jakob läßt das Schiff zu Wasser.'

### Part of Spech tagging and Dependency Parsing

In [118]:
nlp = stanza.Pipeline(
    lang='de',
    processors='tokenize,mwt,pos,lemma,depparse'
)

2025-08-19 16:13:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-19 16:13:31 INFO: Downloaded file to /home/jstet/stanza_resources/resources.json
2025-08-19 16:13:32 INFO: Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-08-19 16:13:32 INFO: Using device: cpu
2025-08-19 16:13:32 INFO: Loading: tokenize
2025-08-19 16:13:32 INFO: Loading: mwt
2025-08-19 16:13:32 INFO: Loading: pos
2025-08-19 16:13:33 INFO: Loading: lemma
2025-08-19 16:13:39 INFO: Loading: depparse
2025-08-19 16:13:39 INFO: Done loading processors!


In [119]:
def display(doc):
    for sent in doc.sentences:
        for word in sent.words:
            print(
                f"{word.text}\t" 
                f"{word.lemma}\t"
                f"{word.upos}\t" # https://universaldependencies.org/u/pos/index.html
                f"{word.xpos}\t" # treebank-specific POS 
                f"{word.feats or '_'}\t" # https://universaldependencies.org/u/feat/index.html
                f"{word.deprel}" # https://universaldependencies.org/u/dep/index.html
            )

Works perfectly for correct spelling.

In [120]:
correct = "Er baute es auf. Er hat es aufgebaut"
correct_doc = nlp(correct)
display(correct_doc)

Er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	nsubj
baute	bauen	VERB	VVFIN	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	root
es	es	PRON	PPER	Case=Acc|Gender=Neut|Number=Sing|Person=3|PronType=Prs	obj
auf	auf	ADP	PTKVZ	_	compound:prt
.	.	PUNCT	$.	_	punct
Er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	nsubj
hat	haben	AUX	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	aux
es	es	PRON	PPER	Case=Acc|Gender=Neut|Number=Sing|Person=3|PronType=Prs	obj
aufgebaut	aufbauen	VERB	VVPP	VerbForm=Part	root


Does not work for properly for incorrect spelling.

In [121]:
misspelled1 = """Herr Jakob so heiß die Haand Figure. Herr Jakob ist neben drei männer und die kuken auf dass wasser.
"""
display(nlp(misspelled1))


Herr	Herr	NOUN	NN	Case=Nom|Gender=Masc|Number=Sing	compound
Jakob	Jakob	PROPN	NE	Case=Nom|Gender=Masc|Number=Sing	nsubj
so	so	ADV	ADV	_	advmod
heiß	heiß	ADJ	ADJD	Degree=Pos	root
die	der	DET	ART	Case=Nom|Definite=Def|Gender=Fem|Number=Sing|PronType=Art	det
Haand	Haand	PROPN	NE	Case=Nom|Gender=Fem|Number=Sing	nsubj
Figure	Figure	PROPN	NE	Case=Nom|Gender=Fem|Number=Sing	flat
.	.	PUNCT	$.	_	punct
Herr	Herr	NOUN	NN	Case=Nom|Gender=Masc|Number=Sing	compound
Jakob	Jakob	PROPN	NE	Case=Nom|Gender=Masc|Number=Sing	nsubj
ist	sein	AUX	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	root
neben	neben	ADP	APPR	_	case
drei	drei	NUM	CARD	NumType=Card	nummod
männer	mann	NOUN	NN	Case=Dat|Gender=Masc|Number=Plur	obl
und	und	CCONJ	KON	_	cc
die	der	DET	ART	Case=Nom|Definite=Def|Number=Plur|PronType=Art	det
kuken	kuken	NOUN	NN	Case=Nom|Number=Plur	conj
auf	auf	ADP	APPR	_	case
dass	dass	SCONJ	KOUS	_	mark
wasser	wassern	ADJ	ADJD	Degree=Pos	conj
.	.	PUNCT	$.	_	punct


In [122]:
misspelled2 = """Und Herr Jakob hat ein Hund, er geit wek."""
display(nlp(misspelled2))

Und	und	CCONJ	KON	_	cc
Herr	Herr	NOUN	NN	Case=Nom|Gender=Masc|Number=Sing	compound
Jakob	Jakob	PROPN	NE	Case=Nom|Gender=Masc|Number=Sing	nsubj
hat	haben	VERB	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	root
ein	ein	DET	ART	Case=Acc|Definite=Ind|Gender=Masc|NumType=Card|Number=Sing|PronType=Art	det
Hund	Hund	NOUN	NN	Case=Acc|Gender=Masc|Number=Sing	obj
,	,	PUNCT	$,	_	punct
er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	nsubj
geit	geien	VERB	VVFIN	Mood=Ind|Number=Sing|Person=3|VerbForm=Fin	conj
wek	wek	ADJ	ADJD	Degree=Pos	xcomp
.	.	PUNCT	$.	_	punct


#### Spacy

In [135]:
nlp = init_parser("de_core_news_lg",
                  "spacy",
                  include_headers=True)

doc = nlp("Er baute es auf. Er hat es aufgebaut")

conll = doc._.conll_str
print(conll)

# sent_id = 1
# text = Er baute es auf.
1	Er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	2	sb	_	_
2	baute	bauen	VERB	VVFIN	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	0	ROOT	_	_
3	es	es	PRON	PPER	Case=Acc|Gender=Neut|Number=Sing|Person=3|PronType=Prs	2	oa	_	_
4	auf	auf	ADP	PTKVZ	_	2	svp	_	SpaceAfter=No
5	.	--	PUNCT	$.	_	2	punct	_	_

# sent_id = 2
# text = Er hat es aufgebaut
1	Er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	2	sb	_	_
2	hat	haben	AUX	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	ROOT	_	_
3	es	es	PRON	PPER	Case=Acc|Gender=Neut|Number=Sing|Person=3|PronType=Prs	4	oa	_	_
4	aufgebaut	aufbauen	VERB	VVPP	VerbForm=Part	2	oc	_	SpaceAfter=No



### Counting VERB

In [ ]:
def count_verb(tagparsed):
    count = 0
    verbs = []
    for send in tagparsed.sentences:
        for word in send.words:
            if word.upos == "VERB":
                count +=1
                verbs.append(word)
    return count,verbs

# using correct example from above
count_verb(correct_doc)

(2,
 [{
    "id": 2,
    "text": "baute",
    "lemma": "bauen",
    "upos": "VERB",
    "xpos": "VVFIN",
    "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "start_char": 3,
    "end_char": 8
  },
  {
    "id": 4,
    "text": "aufgebaut",
    "lemma": "aufbauen",
    "upos": "VERB",
    "xpos": "VVPP",
    "feats": "VerbForm=Part",
    "head": 0,
    "deprel": "root",
    "start_char": 27,
    "end_char": 36
  }])

In [ ]:
temp = "Dieser Satz könnte vier Verben haben, wenn sich das Wetter morgen nicht verschlechtert und ich deswegen kein Fahrrad fahren kann."
count, verbs = count_verb(nlp(temp))
print("Count 1:", count)
temp = "Dieser Satz hätte drei Verben, wenn er nicht bald enden würde"
temp_doc = nlp(temp)
count, verbs = count_verb(nlp(temp_doc))
print("Count 2:", count)

Count 1: 3
Count 2: 2


### Implementing spelling correction with LLMs

In [ ]:
class SpellingCorrection(dspy.Signature):
    """Correct Spelling of german text. Try to correct word for word (but taking into account grammatical correctness based on the whole sentence) and stay true to the original word order."""
    text: str = dspy.InputField()
    corrected: list = dspy.OutputField(desc="The potentially correct words and punctuation as a list in the order as they appear in the sentence.")

examples = [
    dspy.Example(text="Und Herr Jakob hat ein Hund, er geit wek.",corrected=['Und', 'Herr', 'Jakob', 'hat', 'einen', 'Hund', ',', 'er', 'geht', 'weg']).with_inputs("text"),
    dspy.Example(text="Herr Jakob so heiß die Haand Figure. Herr Jakob ist neben drei männer und die kuken auf dass wasser", 
                 corrected=['Herr', 'Jakob', 'heißt', 'die', 'Hauptfigur', ".", 'Herr', 'Jakob', 'ist', 'neben', 'drei', 'Männern', 'und', 'sie', 'gucken', 'auf', 'das', 'Wasser.']).with_inputs("text"),
    ]

lm = dspy.LM("openai/mistralai/mistral-medium-3.1", api_key=os.getenv("OR_KEY"), api_base="https://openrouter.ai/api/v1")
dspy.configure(lm=lm)
prog = dspy.Predict(SpellingCorrection)
teleprompter = dspy.LabeledFewShot()
opt = teleprompter.compile(prog,trainset=examples)


Afterwards, spelling correction, pos tagging and dependency parsing works.

In [ ]:
corrected = " ".join(opt(text="Er hat geöffen. und danach er hat gebaut Schienen und mit ein zug").corrected)
temp_doc = nlp(corrected)
display(temp_doc)

Er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	nsubj
hat	haben	AUX	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	aux
geöffnet	öffnen	VERB	VVPP	VerbForm=Part	root
.	.	PUNCT	$.	_	punct
Und	und	CCONJ	KON	_	cc
danach	danach	ADV	PAV	_	advmod
hat	haben	AUX	VAFIN	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	aux
er	er	PRON	PPER	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	nsubj
Schienen	Schiene	NOUN	NN	Case=Acc|Gender=Fem|Number=Plur	obj
gebaut	bauen	VERB	VVPP	VerbForm=Part	root
und	und	CCONJ	KON	_	cc
mit	mit	ADP	APPR	_	case
einem	ein	DET	ART	Case=Dat|Definite=Ind|Gender=Masc|NumType=Card|Number=Sing|PronType=Art	det
Zug	Zug	NOUN	NN	Case=Dat|Gender=Masc|Number=Sing	obl


In [ ]:
count_verb(temp_doc)

(2,
 [{
    "id": 3,
    "text": "geöffnet",
    "lemma": "öffnen",
    "upos": "VERB",
    "xpos": "VVPP",
    "feats": "VerbForm=Part",
    "head": 0,
    "deprel": "root",
    "start_char": 7,
    "end_char": 15
  },
  {
    "id": 6,
    "text": "gebaut",
    "lemma": "bauen",
    "upos": "VERB",
    "xpos": "VVPP",
    "feats": "VerbForm=Part",
    "head": 0,
    "deprel": "root",
    "start_char": 45,
    "end_char": 51
  }])